In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import roc_auc_score


Importing the datasets

In [2]:
df_train = pd.read_csv('training_set_features.csv')
df_train_label = pd.read_csv('training_set_labels.csv')
df_test = pd.read_csv('test_set_features.csv')


Data Pre-Processing

In [172]:
df_train.set_index('respondent_id').info()

<class 'pandas.core.frame.DataFrame'>
Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   xyz_concern                  26615 non-null  float64
 1   xyz_knowledge                26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_xyz              24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker        

In [3]:
df_train.iloc[:,21:].employment_occupation.value_counts()

employment_occupation
xtkaffoo    1778
mxkfnird    1509
emcorrxb    1270
cmhcxjea    1247
xgwztkwe    1082
hfxkjkmi     766
qxajmpny     548
xqwwgdyp     485
kldqjyjy     469
uqqtjvyb     452
tfqavkke     388
ukymxvdu     372
vlluhbov     354
oijqvulv     344
ccgxvspp     341
bxpfxfdn     331
haliazsg     296
rcertsgn     276
xzmlyyjv     248
dlvbwzss     227
hodpvpew     208
dcjcmpih     148
pvmttkik      98
Name: count, dtype: int64

In [5]:
df_train=df_train.fillna(df_train.mode().iloc[0])
df_train.isnull().sum()

respondent_id                  0
xyz_concern                    0
xyz_knowledge                  0
behavioral_antiviral_meds      0
behavioral_avoidance           0
behavioral_face_mask           0
behavioral_wash_hands          0
behavioral_large_gatherings    0
behavioral_outside_home        0
behavioral_touch_face          0
doctor_recc_xyz                0
doctor_recc_seasonal           0
chronic_med_condition          0
child_under_6_months           0
health_worker                  0
health_insurance               0
opinion_xyz_vacc_effective     0
opinion_xyz_risk               0
opinion_xyz_sick_from_vacc     0
opinion_seas_vacc_effective    0
opinion_seas_risk              0
opinion_seas_sick_from_vacc    0
age_group                      0
education                      0
race                           0
sex                            0
income_poverty                 0
marital_status                 0
rent_or_own                    0
employment_status              0
hhs_geo_re

In [6]:
df_train.income_poverty.value_counts()

income_poverty
<= $75,000, Above Poverty    17200
> $75,000                     6810
Below Poverty                 2697
Name: count, dtype: int64

In [7]:
ordinal_cols = {'xyz_concern': [0, 1, 2, 3],
    'xyz_knowledge': [0, 1, 2],
    'opinion_xyz_vacc_effective': [1, 2, 3, 4, 5],
    'opinion_xyz_risk': [1, 2, 3, 4, 5],
    'opinion_xyz_sick_from_vacc': [1, 2, 3, 4, 5],
    'opinion_seas_vacc_effective': [1, 2, 3, 4, 5],
    'opinion_seas_risk': [1, 2, 3, 4, 5],
    'opinion_seas_sick_from_vacc': [1, 2, 3, 4, 5],
    'age_group': ['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years'],
    'education': ['< 12 Years', '12 Years', 'Some College', 'College Graduate'],
    'income_poverty': ['Below Poverty', '<= $75,000, Above Poverty', '> $75,000']}

nominal_cols = [
    'race', 'sex', 'marital_status', 'rent_or_own', 'employment_status',
    'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']
ordinal_transformer = Pipeline(steps=[

('ordinal', OrdinalEncoder(categories=[ordinal_cols[col] for col in ordinal_cols]))
])

nominal_transformer = Pipeline(steps=[
    
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
transformers=[('ordinal', ordinal_transformer, list(ordinal_cols.keys())),('nominal', nominal_transformer, nominal_cols)],
    remainder='passthrough' )

    
    
    



In [25]:

# Split the data into train and test sets
X_train = df_train.drop(columns=['respondent_id'])
y_train = df_train_label[['xyz_vaccine', 'seasonal_vaccine']]


svm = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MultiOutputClassifier(SVC(kernel='linear', probability=True)))
])

svm.fit(X_train, y_train)




C:\Users\Aayush Bhat\anaconda3\envs\tf\lib\site-packages\sklearn\compose\_column_transformer.py:1624: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinal',
                                                  Pipeline(steps=[('ordinal',
                                                                   OrdinalEncoder(categories=[[0,
                                                                                               1,
                                                                                               2,
                                                                                               3],
                                                                                              [0,
                                                                                               1,
                                                                                               2],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              [1,
                                                                                               2,
                                                                                               3,
                                                                                               4,
                                                                                               5],
                                                                                              ['18 '
                                 

In [26]:
df_test=df_test.fillna(df_test.mode().iloc[0])
X_test =df_test.drop(columns=['respondent_id'])
y_pred = svm.predict_proba(X_test)
y_pred = np.array([pred[:, 1] for pred in y_pred]).T  

In [27]:
submission_dataset= pd.DataFrame({'respondent_id': df_test.iloc[:,0],'xyz_vaccine':y_pred[:,0],'seasonal_vaccine':y_pred[:,1]})


In [28]:
submission_dataset.to_csv('submission_1.csv')